In [5]:
import xarray as xr
import xgcm
import numpy as np
import xroms
import matplotlib.pyplot as plt

# How to calculate with `xarray` and `xroms`

Here we demonstrate a number of calculations built into `xroms`, through accessors to `DataArrays` and `Datasets`.

## `xarray` Datasets
Use an `xarray` accessor in `xroms` to easily perform calculations with syntax 

    ds.xroms.[method]
    
Importantly, the `xroms` accessor to a `Dataset` is initialized with an `xgcm` grid object, stored at `ds.xroms.grid`, which is used to perform the basic grid calculations. More on this under "How to set up grid" below.

The built-in native calculations are properties of the `xroms` accessor and are not functions.

The accessor functions can take in the horizontal then vertical grid label you want the calculation to be on as options:

    ds.xroms.ddz('u', hcoord='rho', scoord='s_rho')  # to make sure result is on rho horizontal grid and s_rho vertical grid, a function
    
or

    ds.xroms.dudz  # return on native grid it is calculated on, a property

Other inputs are available for functions when the calculation involves a derivative and there is a choice for how to treat the boundary (`hboundary` and `hfill_value` for horizontal calculations and `sboundary` and `sfill_value` for vertical calculations). More information on those inputs can be found in the docs for `xgcm` such as under found under:

    ds.xroms.grid.interp?


## `xarray` DataArrays

A few of the more basic methods in `xroms` are available to `DataArrays` too. If the Dataset was read in using an `xroms` load function, any of the `DataArrays` has its grid stored in `da.attrs['grid']`. So, the built-in `xroms` methods for `DataArrays` do not require the grid object to be input.

    ds.temp.xroms.to_grid(hcoord='psi', scoord='s_w')

## Attributes

The intent of `xroms` is to be use `xarray` Datasets and DataArrays for calculations. `xroms` provides attributes as metadata to track calculations, provide context, and to be used as indicators for plots. The `xgcm` grid object that is used in many calculations is also passed around as an attribute in DataArrays.

The option to always keep attributes in `xarray` is turned on in the call to `xroms` because the functionality is assumed.

## `cf-xarray`

Some functionality is added by using the package `cf-xarray`. Necessary attributes are added to datasets read in with `open_netcdf` and `open_zarr` and added in relevant functions.

### Load in data

More information at in [input/output notebook](io.ipynb)

In [6]:
# Helpful to have local file for this
url = 'http://barataria.tamu.edu:8080/thredds/fileServer/forecast_latest/txla2_his_a_latest.nc'
fname = url.split('/')[-1]
chunks = {'ocean_time': 1}

import os
if os.path.exists(fname):
    ds = xroms.open_netcdf(fname, chunks=chunks)
else:
    from subprocess import call
    call(['wget', url])
    ds = xroms.open_netcdf(fname, chunks=chunks)
    
# Alternatively, use file through thredds:
# loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/forecast_latest/txla2_his_f_latest.nc'
# chunks = {'ocean_time':1}
# ds = xr.open_dataset(loc, chunks=chunks)

## `xgcm` grid and extra ROMS coordinates

### How to set up grid

The package `xcgm` has many nice grid functions for ROMS users, however, a bit of set up is required to connect from ROMS to MITgcm output. This grid set up does that.

The `grid` object contains metrics (X, Y, Z) with distances for each grid ('dx', 'dx_u', 'dx_v', 'dx_psi', and 'dz', 'dz_u', 'dz_v', 'dz_w', 'dz_w_u', 'dz_w_v', 'dz_psi', 'dz_w_psi'), and all of these as grid coordinates too. 

Coordinates are added to the dataset when `xroms.open_netcdf`, `xroms.open_mfnetcdf`, or `xroms.open_zarr` are used and the grid object is stored in the `DataArray`  attributes at `da.attrs['grid']`, and in the Dataset through the accessor (`ds.xroms.grid`).

You can also explicitly set up the grid with:

> ds, grid = xroms.roms_dataset(ds)

This process sets up all sorts of useful coordinate variables.

### Grid lengths

Distances between grid nodes on every ROMS grid are calculated in the setup process using `xroms` (that is, by reading in your output using as `xroms` load function or subsequently running `xroms.roms_dataset`). 

* Horizontal grids:
 * inverse distances between nodes are also given in an analogous way (*i.e.*, `ds.pm` and `ds.pn_psi`)
 * distances between nodes are given in meters by dx's and dy's stored in ds, such as: `ds.dx` for the `rho` grid and `ds.dy_psi` for the `psi` grid, calculated from inverse distances
* Vertical grids:
 * There are lazily-evaluated z-coordinates for both `rho` and `w` vertical grids for each horizontal grid.
 * There are also arrays of z distances between nodes, called dz's, available for each combination of grids. For example, there is `ds.dz_u` for z distances on the `u` horizontal and `rho` vertical grid, and there is `ds.dz_w_v` for z distances on the `v` horizontal and `w` vertical grid. These are `[ocean_time x s_* x eta_* x xi_*]` arrays.
 * Arrays of z distances relative to a sea level of 0 are also available. They have analogous names to the previous entries but with "0" on the end. They are computationally faster to use because they do not vary in time. They are also less accurate for this reason but it depends on your use as to how much that matters.

### Grid areas

* Horizontal
  * rho grid `ds.dA`, psi grid `ds.dA_psi`, u grid `ds.dA_u`, v grid `ds.dA_v`
* Vertical
  * These aren't built in but can easily be calculated. For example, for cell areas in the x direction on the rho horizontal and rho vertical grids: `ds.dx * ds.dz`.

### Grid volumes

Time varying: All 8 combinations of 4 horizontal grids and 2 vertical grids are available, such as: `ds.dV` (rho horizontal, rho vertical), and `ds.dV_w_v` (w vertical, v horizontal). 

A user can easily calculate the same but for time-constant dz's, for example as: 

    ds['dV_w'] = ds.dx * ds.dy * ds.dz_w0  # w vertical, rho horizontal, constant in time
    
To calculate the full domain volume in time, you can do:

In [ ]:
ds.dV.sum(('s_rho', 'eta_rho', 'xi_rho'));

In [ ]:
ds.dV.cf.sum(('Z', 'Y', 'X'));  # with cf-xarray accessor

## Change grids

A ROMS user frequently needs to move between horizontal and vertical grids, so it is built into many of the function wrappers, but you can also do it as a separate function. It can also be done to both `Datasets` and `DataArrays` with slightly different syntax. Here we change salinity from its default grids to be on the psi grid horizontally and the s_w grid vertically:

In [ ]:
ds.xroms.to_grid('salt', 'psi', 's_w');   # Dataset
ds.salt.xroms.to_grid('psi', 's_w');   # DataArray

You can also go to the original `xroms` function and avoid the `xarray` accessor if you prefer, though the point of the accessor approach (that is, `ds.xroms...`) is to be easier to remember and less code to write generally.

In [ ]:
xroms.to_grid(ds.salt, ds.salt.attrs['grid'], hcoord='psi', scoord='s_w');

In [ ]:
xroms.to_grid(ds.salt, ds.salt.attrs['grid'], hcoord='rho', scoord='w');

## Dimension ordering convention

By convention, ROMS DataArrays should be in the order ['T', 'Z', 'Y', 'X'], for however many of these dimensions they contain. The following function does this for you:

In [ ]:
xroms.order(ds.temp);
ds.temp.xroms.order();  # accessor

## Basic computations

These are all functions.

### `xarray`

Many [computations](http://xarray.pydata.org/en/stable/computation.html) are built into `xarray` itself. Often it is possible to input the dimension over which to perform a computation by name, such as:

    arr.sum(dim="xi_rho")

or 

    arr.sum(dim=("xi_rho","eta_rho"))

Note that many basic `xarray` calculations should be used with caution when using with ROMS output, since a ROMS grid can be stretched both horizontally and vertically. When using these functions, consider if your calculation should account for variable grid cell distances, areas, or volumes. Additionally, it is straight-forward to use basic grid functions from `xarray` on a ROMS time dimension (resampling, differentiation, interpolation, etc), however, be careful before using these functions on spatial dimensions for the same reasons as before.

In [ ]:
ds.salt.mean(dim=("xi_rho","eta_rho"));

`cf-xarray`

In [ ]:
ds.salt.cf.mean(("Y","X"));

#### `xroms` grid-based metrics

Spatial metrics that account for the variable grid cell sizing in ROMS (both curvilinear horizontal and s vertical) are available by wrapping `xgcm` functions. These also have the additional benefit that the user can change grids and attributes are tracked. The available functions are:

* gridsum
* gridmean

Example usage:

    xroms.gridsum(ds.temp, grid, dim)

    ds['temp'].xroms.gridsum(dim)

where dimension names in the `xgcm` convention are 'Z', 'Y', or 'X'. `dim` can be a string, list, or tuple of combinations of these names for dimensions to average over.

sum

In [ ]:
ds.u.xroms.gridsum(('Y','Z'));

mean

In [ ]:
ds.v.xroms.gridmean(('Y','X'));

## Derivatives

### Vertical

In [ ]:
ds.xroms.ddz('salt');  # Dataset

In [ ]:
ds.salt.xroms.ddz();  # DataArray

In [ ]:
xroms.ddz(ds.salt, ds.salt.attrs['grid']);  # No accessor

There are more options available for this call:

In [ ]:
ds.xroms.ddz('salt', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # Dataset

In [ ]:
ds.salt.xroms.ddz(hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # DataArray

In [ ]:
xroms.ddz(ds.salt, ds.salt.attrs['grid'], hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # No accessor

### Horizontal

In [ ]:
ds.xroms.ddxi('u');  # horizontal xi-direction gradient
ds.xroms.ddeta('u');  #  horizontal eta-direction gradient

In [ ]:
ds.u.xroms.ddxi();
ds.u.xroms.ddeta();

In [ ]:
dtempdxi, dtempdeta = xroms.hgrad(ds.temp, ds.temp.attrs['grid'])

In [ ]:
xroms.ddxi(ds.temp, ds.temp.attrs['grid']);
xroms.ddeta(ds.temp, ds.temp.attrs['grid']);

### Time

Use `xarray` directly for this.

In [ ]:
ddt = ds.chunk({'ocean_time': 2}).differentiate('ocean_time', datetime_unit='s')

## Built-in Physical Calculations

These are all properties of the accessor, so should be called without (). Demostrated below are the calculations using the accessor and not using the accessor.

### Horizontal speed

In [ ]:
ds.xroms.speed;

In [ ]:
xroms.speed(ds.u, ds.v, ds.u.attrs['grid']); 

### Kinetic energy

In [ ]:
ds.xroms.KE;

In [ ]:
speed = xroms.speed(ds.u, ds.v, ds.u.attrs['grid']);
xroms.KE(ds.rho0, speed);

### Geostrophic velocities

In [ ]:
ds.xroms.ug;
ds.xroms.vg;

In [ ]:
ug, vg = xroms.uv_geostrophic(ds.zeta, ds.f, ds.zeta.attrs['grid']);

### Eddy kinetic energy (EKE)

In [ ]:
ds.xroms.EKE;

In [ ]:
ug, vg = xroms.uv_geostrophic(ds.zeta, ds.f, ds.zeta.attrs['grid']);
xroms.EKE(ug, vg, ug.attrs['grid']);

### Vertical shear

Since it is a common use case, there are specific methods to return the u and v components of vertical shear on their own grids. These are just available for Datasets.

In [ ]:
ds.xroms.dudz;
ds.xroms.dvdz;

In [ ]:
xroms.dudz(ds.u, ds.u.attrs['grid']);
xroms.dvdz(ds.v, ds.v.attrs['grid']);

If we want to calculate something with both, we need them on the same grid. For this, we can input the desired resultant grid:

In [ ]:
ds.xroms.dudz.xroms.to_grid(hcoord='rho', scoord='s_rho')**2 + ds.xroms.dvdz.xroms.to_grid(hcoord='rho', scoord='s_rho')**2;

Or in this case, the magnitude of the vertical shear is also a built-in derived variable for the `xroms` accessor:

In [ ]:
ds.xroms.vertical_shear;

In [ ]:
dudz = xroms.dudz(ds.u, ds.u.attrs['grid']);
dvdz = xroms.dvdz(ds.v, ds.v.attrs['grid']);
xroms.vertical_shear(dudz, dvdz, dudz.attrs['grid']);

### Vertical vorticity

In [ ]:
ds.xroms.vort;

In [ ]:
xroms.relative_vorticity(ds.u, ds.v, ds.u.attrs['grid']);

### Ertel potential vorticity

The accessor assumes you want the Ertel potential vorticity of the buoyancy:

In [ ]:
ds.xroms.ertel;

In [ ]:
sig0 = xroms.potential_density(ds.temp, ds.salt);
buoyancy = xroms.buoyancy(sig0, rho0=ds.rho0);
xroms.ertel(buoyancy, ds.u, ds.v, ds.f, ds.u.attrs['grid'], scoord='s_w');

Alternatively, the user can access the original function and use a different tracer for this calculation, and can return the result on a different vertical grid, for example:

In [ ]:
xroms.ertel(ds.dye_01, ds.u, ds.v, ds.f, ds.u.attrs['grid'], scoord='s_w');

### Density

In [ ]:
ds.xroms.rho;

In [ ]:
xroms.density(ds.temp, ds.salt);

### Potential density

In [ ]:
ds.xroms.sig0;

In [ ]:
xroms.potential_density(ds.temp, ds.salt);

### Buoyancy

In [ ]:
ds.xroms.buoyancy;

In [ ]:
sig0 = xroms.potential_density(ds.temp, ds.salt);
xroms.buoyancy(sig0);

### Buoyancy frequency

Also called vertical buoyancy gradient.

In [ ]:
ds.xroms.N2;

In [ ]:
rho = xroms.density(ds.temp, ds.salt)  # calculate rho if not in output
xroms.N2(rho, rho.attrs['grid']);

### Horizontal buoyancy gradient 

In [ ]:
ds.xroms.M2;

In [ ]:
rho = xroms.density(ds.temp, ds.salt)  # calculate rho if not in output
xroms.M2(rho, rho.attrs['grid']);

### Mixed layer depth

This is not a property since the threshold is a parameter.

In [ ]:
ds.xroms.mld(thresh=0.03);

In [ ]:
sig0 = xroms.potential_density(ds.temp, ds.salt);
xroms.mld(sig0, ds.h, ds.mask_rho, thresh=0.03);

## Time-based calculations including climatologies

### Rolling averages in time

Here is an example of computing a rolling average in time.

More information about rolling operations [is available](http://xarray.pydata.org/en/stable/computation.html#rolling-window-operations).

In [ ]:
roll = ds.salt.rolling(ocean_time=3, center=True, min_periods=2).mean()
roll.isel(s_rho=-1, eta_rho=50, xi_rho=40).plot()
ds.salt.isel(s_rho=-1, eta_rho=50, xi_rho=40).plot()

### Resampling in time

Can't have any chunks in the time dimension to do this. More info: http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html

#### Upsample

Upsample to a higher resolution in time. Makes sense to interpolate to fill in data when upsampling, but can also forward or backfill, or just add nan's.

In [ ]:
dstest = ds.resample(ocean_time='30min', restore_coord_dims=True, keep_attrs=True).interpolate()

Plot to visually inspect results

In [ ]:
ds.salt.cf.isel(Y=30, X=20, Z=-1, T=slice(0,10)).plot(marker='o')
dstest.salt.cf.isel(Y=30, X=20, Z=-1).isel(ocean_time=slice(0,20)).plot(marker='x')

#### Downsample

Resample down to lower resolution in time. This requires appending a method to aggregate the extra data, such as a `mean`. Note that other options can be used to shift the result within the interval of aggregation in various ways.

In [ ]:
dstest = ds.resample(ocean_time='3H').mean()

In [ ]:
ds.salt.isel(eta_rho=30, xi_rho=20, s_rho=-1, ocean_time=slice(0,10)).plot(marker='o')
dstest.salt.isel(eta_rho=30, xi_rho=20, s_rho=-1, ocean_time=slice(0,5)).plot(marker='x')

#### Seasonal average, over time

This is an example of [resampling](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html).

    da.cf.resample({'T': [time frequency string]}).reduce([aggregation function])
    
For example, calculate the mean temperature every quarter in time with the following:

    ds.temp.cf.resample({'T': 'QS'}).reduce(np.mean)

or the aggregation function can be appended on the end directly with:

    ds.temp.cf.resample({'T': 'QS'}).mean()

The result of this calculation is a time series of downsampled chunks of output in time, the frequency of which is selected by input "time frequency string", and aggregated by input "aggregation function".

Examples of the time frequency string are:
* "QS": quarters, starting in January of each year and averaging three months.
  * Also available are selections like "QS-DEC", quarters but starting with December to better align with seasons. Other months are input options as well.
* "MS": monthly
* "D": daily
* "H": hourly
* Many more options are given [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).

Examples of aggregation functions are:
* np.mean
* np.max
* np.min
* np.sum
* np.std

Result of downsampling a 4D salt array from hourly to 6-hourly, for example, gives: `[ocean_time x s_rho x eta_rho x xi_rho]`, where `ocean_time` has about 1/6 of the number of entries reflecting the aggregation in time.

In [ ]:
ds.temp.cf.resample(indexer={'T': '6H'}).reduce(np.mean);

Use the following to calculate a seasonal or quarterly average over time (noting that this dataset has only a day worth of output so there is only 1 quarter):

In [ ]:
ds.temp.cf.resample(indexer={'T': 'QS'}).reduce(np.mean);

### Seasonal mean over all available time

This is how to average over the full dataset period by certain time groupings using xarray `groupby` which is like pandas version. In this case we show the seasonal mean averaged across the full model time period. The syntax for this is:

    da.salt.cf.groupby('T.[time string]').reduce([aggregation function])

For example, to average salt by season:

    da.salt.cf.groupby('T.season').reduce(np.mean)

or

    da.salt.cf.groupby('T.season').mean()

Options for the time string include:
* 'season'
* 'year'
* 'month'
* 'day'
* 'hour'
* 'minute'
* 'second'
* 'dayofyear'
* 'week'
* 'dayofweek'
* 'weekday'
* 'quarter'

More information about options for time (including "derived" datetime coordinates) is [here](https://xarray.pydata.org/en/v0.16.0/time-series.html#datetime-components).

Examples of aggregation functions are:
* np.mean
* np.max
* np.min
* np.sum
* np.std

Result of averaging over seasons for a 4D salt array returns, for example: `[season x s_rho x eta_rho x xi_rho]`, where `season` has 4 entries, each covering 3 months of the year.

In [ ]:
# this example has only 1 season because it is a short example file
ds.temp.cf.groupby('T.season').mean();